[![Binder](https://mybinder.org/badge_logo.svg)](https://lab.mlpack.org/v2/gh/mlpack/examples/master?urlpath=lab%2Ftree%2Fdominant-colors-with-kmeans%2Fdominant-colors-kmeans-cpp.ipynb)

In [1]:
/**
 * @file dominant-colors-kmeans-cpp.ipynb
 *
 * A simple example usage of K-means clustering
 * to find the most dominant colors in an image.
 *
 * The dominant colors are colors that are represented
 * most in the image.
 */

Download some example images.

In [2]:
!wget -q -O jurassic-park.png https://datasets.mlpack.org/jurassic-park.png

In [3]:
!wget -q -O the-godfather.png https://datasets.mlpack.org/the-godfather.png

In [4]:
!wget -q -O the-grand-budapest-hotel.png https://datasets.mlpack.org/the-grand-budapest-hotel.png

In [5]:
#include <mlpack/xeus-cling.hpp>
// Enable image load/save support.
#define HAS_STB
#include <mlpack.hpp>
#include <sstream>

In [6]:
// Header files to create and show images.
#include "xwidgets/ximage.hpp"
#include "../utils/stackedbar.hpp"

In [7]:
using namespace mlpack;

In [8]:
// Before we apply K-means on an image we have to be aware that the RGB color space has some shortages. In fact, it's
// tempting to simply compare the euclidean distance difference between the red, green, and blue aspects of an RGB.
// Unfortunately RGB was intended for convenient use with electronic systems, so is not very similar to average human
// perception. Applying K-means using the euclidean distance quickly reveals sporadic and often drastically different
// results than one would expect of visually similar colors.  There are several ways to tackle the issue and to calculate
// the perceived difference in color. The most popular method is known as CIE 1976, or more commonly just CIE76. This
// method uses the Euclidean distance, however, the trick is to first convert to the CIE*Lab color space.

// Function to convert RGB into CIE*Lab color space.
void rgb2lab(const double R,
             const double G,
             const double B,
             double& ls,
             double& as,
             double& bs )
{
    double varR = R / 255.0;
    double varG = G / 255.0;
    double varB = B / 255.0;

    if (varR > 0.04045)
        varR = std::pow(((varR + 0.055) / 1.055), 2.4 );
    else
        varR /= 12.92;

    if (varG > 0.04045)
        varG = std::pow(((varG + 0.055) / 1.055), 2.4);
    else
        varG /= 12.92;

    if (varB > 0.04045)
        varB = std::pow(((varB + 0.055 ) / 1.055), 2.4);
    else
        varB = varB / 12.92;

    varR *= 100.;
    varG *= 100.;
    varB *= 100.;

    double X = varR * 0.4124 + varG * 0.3576 + varB * 0.1805;
    double Y = varR * 0.2126 + varG * 0.7152 + varB * 0.0722;
    double Z = varR * 0.0193 + varG * 0.1192 + varB * 0.9505;

    double varX = X / 95.047;
    double varY = Y / 100.000;
    double varZ = Z / 108.883;

    if (varX > 0.008856)
        varX = std::pow(varX, 1.0 / 3.0);
    else
        varX = (7.787 * varX) + (16.0 / 116.0);
    
    if (varY > 0.008856)
        varY = std::pow(varY, 1.0 / 3.0);
    else
        varY = (7.787 * varY) + (16.0 / 116.0);
    
    if (varZ > 0.008856)
        varZ = std::pow(varZ, 1.0 / 3.0);
    else
        varZ = (7.787 * varZ) + (16.0 / 116.0);

    ls = (116.0 * varY) - 16.0;
    as = 500.0 * (varX - varY);
    bs = 200.0 * (varY - varZ);
}

In [9]:
// Function to convert CIE*Lab into RGB color space.
void lab2rgb(const double ls,
             const double as,
             const double bs,
             double& R,
             double& G,
             double& B )
{
    double varY = (ls + 16.0) / 116.0;
    double varX = as / 500.0 + varY;
    double varZ = varY - bs / 200.0;

    if (std::pow(varY, 3.0) > 0.008856)
        varY = std::pow(varY, 3.0);
    else
        varY = (varY - 16.0 / 116.0) / 7.787;
    
    if (std::pow(varX, 3.0) > 0.008856)
        varX = std::pow(varX, 3.0);
    else
        varX = (varX - 16.0 / 116.0) / 7.787;
    
    if (std::pow(varZ, 3.0) > 0.008856)
        varZ = std::pow(varZ, 3);
    else
        varZ = (varZ - 16.0 / 116.0) / 7.787;

    double X = 95.047 * varX;
    double Y = 100.000 * varY;
    double Z = 108.883 * varZ;

    varX = X / 100.0;
    varY = Y / 100.0;
    varZ = Z / 100.0;

    double varR = varX * 3.2406 + varY * -1.5372 + varZ * -0.4986;
    double varG = varX * -0.9689 + varY * 1.8758 + varZ * 0.0415;
    double varB = varX * 0.0557 + varY * -0.2040 + varZ * 1.0570;

    if (varR > 0.0031308)
        varR = 1.055 * std::pow(varR, (1.0 / 2.4)) - 0.055;
    else
        varR *= 12.92;
    
    if (varG > 0.0031308)
        varG = 1.055 * std::pow(varG, (1.0 / 2.4)) - 0.055;
    else
        varG *= 12.92;
    if (varB > 0.0031308)
        varB = 1.055 * std::pow(varB, (1.0 / 2.4)) - 0.055;
    else
        varB = 12.92 * varB;

    R = varR * 255.0;
    G = varG * 255.0;
    B = varB * 255.0;
}

In [10]:
// Function to convert RGB matrix into CIE*Lab color space.
void rgb2labMatrix(arma::mat& matrix)
{
    for (size_t i = 0; i < matrix.n_cols; ++i)
    {
        rgb2lab(matrix.col(i)(0),
                matrix.col(i)(1),
                matrix.col(i)(2),
                matrix.col(i)(0),
                matrix.col(i)(1),
                matrix.col(i)(2));
    }
}

In [11]:
// Function to convert CIE*Lab matrix into RGB color space.
void lab2rgbMatrix(arma::mat& matrix)
{
    for (size_t i = 0; i < matrix.n_cols; ++i)
    {
        lab2rgb(matrix.col(i)(0),
                matrix.col(i)(1),
                matrix.col(i)(2),
                matrix.col(i)(0),
                matrix.col(i)(1),
                matrix.col(i)(2));
    }
}

In [12]:
// Helper function to create the color string from the K-means centroids.
void GetColorBarData(std::string& values,
                     std::string& colors,
                     const size_t cluster,
                     const arma::Row<size_t>& assignments,
                     const arma::mat& centroids)
{
    arma::uvec h = arma::histc(arma::conv_to<arma::vec>::from(assignments), arma::linspace<arma::vec>(0, cluster - 1, cluster));
    arma::uvec indices = arma::sort_index(h);

    std::stringstream valuesString;
    std::stringstream colorsString;
    for (size_t i = 0; i < indices.n_elem; ++i)
    {
        colorsString << (int)centroids.col(indices(i))(0) << ";"
                     << (int)centroids.col(indices(i))(1) << ";"
                     << (int)centroids.col(indices(i))(2) << ";";

        valuesString << h(indices(i)) << ";";
    }
    
    values = valuesString.str();
    colors = colorsString.str();
}

In [13]:
// Load the example image.
arma::Mat<unsigned char> imageMatrix;
data::ImageInfo info;
data::Load("jurassic-park.png", imageMatrix, info, false);

In [14]:
// Print the image shape.
std::cout << "Image info -"
          << " Width:" << info.Width()
          << " Height: " << info.Height()
          << " Channels: " << info.Channels() << std::endl;

Image info - Width:600 Height: 450 Channels: 3


In [15]:
// Each column of the image matrix contains an image that
// is vectorized in the format of [R, G, B, R, G, B, ..., R, G, B].
// Here we transform the image data into the expected format:
// [[R, G, B],
//  [R, G, B],
//  ...
//  [R, G, B]]
arma::mat imageData = arma::conv_to<arma::mat>::from(
    arma::reshape(imageMatrix, info.Channels(), imageMatrix.n_elem / 3));

// Remove the alpha channel if the image comes with one.
if (info.Channels() > 3)
    imageData.shed_row(3);

// Convert from RGB to CIE*Lab color space.
rgb2labMatrix(imageData);

In [16]:
// Perform K-means clustering using the Euclidean distance.
//
// For more information checkout https://mlpack.org/doc/stable/doxygen/classmlpack_1_1kmeans_1_1KMeans.html
// or uncomment the line below.
// ?KMeans<>

// The assignments will be stored in this vector.
arma::Row<size_t> assignments;

// The centroids will be stored in this matrix.
arma::mat centroids;

// The number of clusters we are getting (colors).
// For the image we like the see the first 5 dominate colors.
size_t cluster = 5;

// Initialize with the default arguments.
KMeans<> kmeans;
kmeans.Cluster(imageData, cluster, assignments, centroids);

// Convert back from CIE*Lab to RGB color space to plot the result.
lab2rgbMatrix(centroids);

In [17]:
// Show the input image.
auto im = xw::image_from_file("jurassic-park.png").finalize();
im

A Jupyter widget with unique id: 9749727c76dd479db764ab3d3feedceb

In [18]:
// Create color bar data using the centroids matrix and assignments vector.
// In our case which the centroids matrix contains the dominant colors in
// RGB color space, and the assignments vector contains the associated
// dominant color for each pixel in the image.
std::string values, colors;
GetColorBarData(values, colors, cluster, assignments, centroids);

// Show the dominant colors.
StackedBar(values, colors, "jurassic-park-colors.png");
auto im = xw::image_from_file("jurassic-park-colors.png").finalize();
im

A Jupyter widget with unique id: b9be2ffe28314ab5950b4c0a351a15cb

In [19]:
// Load the example image.
arma::Mat<unsigned char> imageMatrix;
data::ImageInfo info;
data::Load("the-godfather.png", imageMatrix, info, false);

In [20]:
// Print the image shape.
std::cout << "Image info -"
          << " Width:" << info.Width()
          << " Height: " << info.Height()
          << " Channels: " << info.Channels() << std::endl;

Image info - Width:376 Height: 500 Channels: 3


In [21]:
// Each column of the image matrix contains an image that
// is vectorized in the format of [R, G, B, R, G, B, ..., R, G, B].
// Here we transform the image data into the expected format:
// [[R, G, B],
//  [R, G, B],
//  ...
//  [R, G, B]]
arma::mat imageData = arma::conv_to<arma::mat>::from(
    arma::reshape(imageMatrix, info.Channels(), imageMatrix.n_elem / 3));

// Remove the alpha channel if the image comes with one.
if (info.Channels() > 3)
    imageData.shed_row(3);

// Convert from RGB to CIE*Lab color space.
rgb2labMatrix(imageData);

In [22]:
// Perform K-means clustering using the Euclidean distance.
//
// For more information checkout https://mlpack.org/doc/stable/doxygen/classmlpack_1_1kmeans_1_1KMeans.html
// or uncomment the line below.
// ?KMeans<>

// The assignments will be stored in this vector.
arma::Row<size_t> assignments;

// The centroids will be stored in this matrix.
arma::mat centroids;

// The number of clusters we are getting (colors).
// For the image we like the see the first 4 dominate colors.
size_t cluster = 4;

// Initialize with the default arguments.
KMeans<> kmeans;
kmeans.Cluster(imageData, cluster, assignments, centroids);

// Convert back from CIE*Lab to RGB color space to plot the result.
lab2rgbMatrix(centroids);

In [23]:
// Show the input image.
auto im = xw::image_from_file("the-godfather.png").finalize();
im

A Jupyter widget with unique id: 1f6e996f8e8c431a823cbe287e0d95a7

In [24]:
// Create color bar data using the centroids matrix and assignments vector.
// In our case which the centroids matrix contains the dominant colors in
// RGB color space, and the assignments vector contains the associated
// dominant color for each pixel in the image.
std::string values, colors;
GetColorBarData(values, colors, cluster, assignments, centroids);

// Show the dominant colors.
StackedBar(values, colors, "the-godfather-colors.png");
auto im = xw::image_from_file("the-godfather-colors.png").finalize();//
im

A Jupyter widget with unique id: e5665e18c30749c7acc1504d5a61381f

In [25]:
// Load the example image.
arma::Mat<unsigned char> imageMatrix;
data::ImageInfo info;
data::Load("the-grand-budapest-hotel.png", imageMatrix, info, false);

In [26]:
// Print the image shape.
std::cout << "Image info -"
          << " Width:" << info.Width()
          << " Height: " << info.Height()
          << " Channels: " << info.Channels() << std::endl;

Image info - Width:913 Height: 475 Channels: 3


In [27]:
// Each column of the image matrix contains an image that
// is vectorized in the format of [R, G, B, R, G, B, ..., R, G, B].
// Here we transform the image data into the expected format:
// [[R, G, B],
//  [R, G, B],
//  ...
//  [R, G, B]]
arma::mat imageData = arma::conv_to<arma::mat>::from(
    arma::reshape(imageMatrix, info.Channels(), imageMatrix.n_elem / 3));

// Remove the alpha channel if the image comes with one.
if (info.Channels() > 3)
    imageData.shed_row(3);

// Convert from RGB to CIE*Lab color space.
rgb2labMatrix(imageData);

In [28]:
// Perform K-means clustering using the Euclidean distance.
//
// For more information checkout https://mlpack.org/doc/stable/doxygen/classmlpack_1_1kmeans_1_1KMeans.html
// or uncomment the line below.
// ?KMeans<>

// The assignments will be stored in this vector.
arma::Row<size_t> assignments;

// The centroids will be stored in this matrix.
arma::mat centroids;

// The number of clusters we are getting (colors).
// For the image we like the see the first 4 dominate colors.
size_t cluster = 4;

// Initialize with the default arguments.
KMeans<> kmeans;
kmeans.Cluster(imageData, cluster, assignments, centroids);

// Convert back from CIE*Lab to RGB color space to plot the result.
lab2rgbMatrix(centroids);

In [29]:
// Show the input image.
auto im = xw::image_from_file("the-grand-budapest-hotel.png").finalize();
im

A Jupyter widget with unique id: d2458ef6cb7e4ca08183dea30e472e64

In [30]:
// Create color bar data using the centroids matrix and assignments vector.
// In our case which the centroids matrix contains the dominant colors in
// RGB color space, and the assignments vector contains the associated
// dominant color for each pixel in the image.
std::string values, colors;
GetColorBarData(values, colors, cluster, assignments, centroids);
StackedBar(values, colors, "the-grand-budapest-hotel-colors.png");

// Show the dominant colors.
auto im = xw::image_from_file("the-grand-budapest-hotel-colors.png").finalize();
im

A Jupyter widget with unique id: 9c3cbdd4aae649f6a533aff949824482